# DeepAsr (DeepAsrNetwork1)

In [ ]:
# !wget http://www.openslr.org/resources/12/train-clean-100.tar.gz

In [ ]:
# !tar xzvf train-clean-100.tar.gz

In [ ]:
# !pip install tensorflow==2.1.0

In [ ]:
# ! pip install deepasr

# 1. Prepare DataSet

In [ ]:
import os, numpy as np
import pandas as pd

In [ ]:
org_path = 'LibriSpeech/train-clean-100/'
count = 0
inp = []
k=0
audio_name = []
audio_trans = []
for dir1 in os.listdir(org_path):
  dir2_path = org_path+dir1+'/'
  #print(dir2_path)
  for dir2 in os.listdir(dir2_path):
    dir3_path = dir2_path+dir2+'/'
    
    for audio in os.listdir(dir3_path):
      if audio.endswith('.txt'):
        k+=1
        file_path = dir3_path + audio
        with open(file_path) as f:
          line = f.readlines()
          for lines in line:
            audio_name.append(dir3_path+lines.split()[0]+'.flac')
            words2 = lines.split()[1:]
            words4=' '.join(words2)
            audio_trans.append(words4)

In [ ]:
# create dataframe
df = pd.DataFrame({"path":audio_name,"transcripts":audio_trans})

In [ ]:
df.shape

(28539, 2)

In [ ]:
# filter transcript less than 100 charcters
train_df = df[df['transcripts'].str.len() < 100]
# train_df = df.sample(n = 3000) 

In [ ]:
train_df.shape

(3194, 2)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import random
from tensorflow.keras.layers import Input,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import deepasr as asr

In [14]:
tf.__version__

'2.1.0'

# 2. Preprocessing

In [ ]:
# get MFCC-FilterBank features of audio
features_extractor = asr.features.FilterBanks(features_num=161, winlen=0.02,winstep=0.025,winfunc=np.hanning)

# get Spectrogram features of audio
# features_extractor = asr.features.Spectrogram(
#             features_num=161,
#             samplerate=16000,
#             winlen=0.02,
#             winstep=0.01,
#             winfunc=np.hanning
#         )

In [ ]:
# get the waveform of audio 
mid_features = list()
audios = list()
transcripts = list()
for row in train_df.values.tolist():
  librosa_features = asr.utils.read_audio(row[0], sample_rate=16000, mono=True)
  if(librosa_features.shape[0]<160000):
    mid_features.append(librosa_features)
    audios.append(row[0])
    transcripts.append(row[1])

In [ ]:
len(audios),len(transcripts)

In [ ]:
# extracts features from audio
trainX = features_extractor(mid_features)

In [ ]:
# get english text features
alphabet_en = asr.vocab.Alphabet(lang='en')

In [ ]:
# extract transcript labels
trainy = alphabet_en.get_batch_labels(transcripts)

In [ ]:
# get input_length and label_length for ctc
label_len = [len(trans) for trans in transcripts]  # length of each transcription
label_lengths = np.array(label_len).reshape(-1, 1)  # reshape to 1d

input_lengths = np.ones((trainy.shape[0], 1)) * trainy.shape[1]
for i in range(trainy.shape[0]):
    input_lengths[i] = trainy.shape[1]  # num of features from labels

In [19]:
trainX.shape,trainy.shape,label_lengths.shape,input_lengths.shape

((3153, 394, 161), (3153, 99), (3153, 1), (3153, 1))

In [ ]:
# model optimizer
optimizer = tf.keras.optimizers.Adam(
        lr=1e-4,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8
    )

In [ ]:
# output sequence decoder 
decoder = asr.decoder.GreedyDecoder()
# decoder = asr.decoder.BeamSearchDecoder(beam_width = 100, top_paths = 1)

In [ ]:
# asr neural network
model = asr.model.get_deepasrnetwork1(
        input_dim=161,
        output_dim=29,
        is_mixed_precision=True
    )

In [ ]:
# compile model with ctcloss
model = asr.model.compile_model(model,optimizer)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, None, 161)]  0                                            
__________________________________________________________________________________________________
BN_1 (BatchNormalization)       (None, None, 161)    644         the_input[0][0]                  
__________________________________________________________________________________________________
Conv1D_1 (Conv1D)               (None, None, 220)    177320      BN_1[0][0]                       
__________________________________________________________________________________________________
CNBN_1 (BatchNormalization)     (None, None, 220)    880         Conv1D_1[0][0]                   
____________________________________________________________________________________________

In [ ]:
# Defining a helper function to save the model after each epoch 
# in which the loss decreases 
filepath = project_path+'checkpoint1/model_weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,save_weights_only=True, mode='min')
# Defining a helper function to reduce the learning rate each time 
# the learning plateaus 
reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2, patience = 1, min_lr = 0.001)
# stop traning if there increase in loss
es = EarlyStopping(monitor='loss', mode='min', verbose=1,patience=2)
callbacks = [checkpoint]

In [ ]:
batch_size = 128

In [ ]:
  # input features dictionary
  inputs = {
    'the_input': trainX,
    'the_labels': trainy,
    'input_length': input_lengths,
    'label_length': label_lengths,
  }
  # ctc output
  outputs = {'ctc': np.zeros([len(trainX)])}

# 3. Model traning

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CollectiveCommunication.AUTO


In [ ]:
with strategy.scope():
    model = model

In [ ]:
model.fit(inputs, outputs,batch_size = batch_size, epochs=500,callbacks=callbacks, verbose =1)

Train on 3153 samples
Epoch 1/500
3072/3153 [============================>.] - ETA: 0s - loss: inf - accuracy: 0.0000e+00
Epoch 00001: loss did not improve from inf
3153/3153 [==============================] - 23s 7ms/sample - loss: inf - accuracy: 0.0000e+00
Epoch 2/500
3072/3153 [============================>.] - ETA: 0s - loss: inf - accuracy: 0.0000e+00
Epoch 00002: loss did not improve from inf
3153/3153 [==============================] - 18s 6ms/sample - loss: inf - accuracy: 0.0000e+00
Epoch 3/500
3072/3153 [============================>.] - ETA: 0s - loss: inf - accuracy: 0.0000e+00
Epoch 00003: loss did not improve from inf
3153/3153 [==============================] - 18s 6ms/sample - loss: inf - accuracy: 0.0000e+00
Epoch 4/500
3072/3153 [============================>.] - ETA: 0s - loss: inf - accuracy: 0.0000e+00
Epoch 00004: loss did not improve from inf
3153/3153 [==============================] - 18s 6ms/sample - loss: inf - accuracy: 0.0000e+00
Epoch 5/500
3072/3153 [===

In [ ]:
# for i in range(3000):
#   samp=random.sample(range(len(trainX)-25),batch_size)
#   x3=[trainX[i] for i in samp]
#   x3 = np.array(x3)
# #   x3 = np.array(x3)/100
# #   #x3 = x3/np.max(x3)
#   y3 = [trainy[i] for i in samp]
#   y3 = np.array(y3)
#   input_lengths2 = [input_lengths[i] for i in samp]
#   label_lengths2 = [label_lengths[i] for i in samp]
#   input_lengths2 = np.array(input_lengths2)
#   label_lengths2 = np.array(label_lengths2)
#   inputs = {
#     'the_input': x3,
#     'the_labels': y3,
#     'input_length': input_lengths2,
#     'label_length': label_lengths2,
#   }

#   outputs = {'ctc': np.zeros([batch_size])}  
#   if(i%100==0):
#     print("Iter:",i)
#     print("input features: ", x3.shape)
#     print("input labels: ", y3.shape)
#     print(input_lengths2.shape)
#     print(label_lengths2.shape)
#     model.fit(inputs, outputs,batch_size = batch_size, epochs=3, verbose =1)
    
#     model.save(project_path+'checkpoint1/model.hdf5')
#     model.save_weights(project_path+'checkpoint1/model_weights.hdf5')
#   else:
#     model.fit(inputs, outputs,batch_size = batch_size, epochs=3, verbose =0)

In [ ]:
# model.save(project_path+'checkpoints/model.hdf5')
model.save_weights(project_path+'checkpoint1/model_weights.hdf5')

# 4. Model testing

In [ ]:
# build mode from trained model input and output layers
model2 = Model(inputs = model.get_layer('the_input').output, outputs = model.get_layer('the_output').output)

In [ ]:
# load trained model weights
model2.load_weights(project_path+'checkpoint1/model_weights.hdf5')

In [ ]:
train_data = pd.DataFrame({"path":audios,"transcripts":transcripts})

In [27]:
train_data.shape

(3153, 2)

In [28]:
# get testing audio and transcript from dataset
index = np.random.randint(train_data.shape[0])
data = train_data.iloc[index]
test_file = data[0]
test_transcript = data[1]
print("Index:",index)
# Audio file
print("Audio File:",test_file)
# ground truth
print("Audio Transcription:", test_transcript)
print("Transcript length:",len(test_transcript))

Index: 1061
Audio File: LibriSpeech/train-clean-100/730/360/730-360-0042.flac
Audio Transcription: NO MOTHER HAD BLESSED ME WITH SMILES AND CARESSES
Trancript length: 49


In [85]:
# get audio features
features = features_extractor.make_features(asr.utils.read_audio(test_file, sample_rate=16000, mono=True))
# Pad features Return the single array (batch_size, time, features)
in_features = features_extractor.align([features], features_extractor.features_shape)
# in_features = np.array([trainX[index]])
in_features.shape

(1, 394, 161)

In [ ]:
# layer_names=[layer.name for layer in model.layers]

In [ ]:
# predict labels
pred= model2.predict(in_features)

In [88]:
pred.shape

(1, 386, 29)

In [ ]:
# decode predicted labels
decoded_labels = decoder(pred,trainy.shape[1])

In [90]:
decoded_labels

[array([14, 15,  0, 13, 15, 20,  8,  5, 18,  0,  8,  1,  4,  0,  2, 12,  5,
        19, 19,  5,  4,  0, 13,  5,  0, 23,  9, 20,  8,  0, 19, 13,  9, 12,
         5, 19,  0,  1, 14,  4,  0,  3,  1, 18,  5, 19, 19,  5, 19])]

In [ ]:
# get transcript from labels
predictions = alphabet_en.get_batch_transcripts(decoded_labels)

In [92]:
# predicted transcript
predictions[0].upper()

'NO MOTHER HAD BLESSED ME WITH SMILES AND CARESSES'

In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))